In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import math
from numba import njit, prange, cuda, vectorize

In [2]:
@njit()
def single_boundary_numba(n, o_x=0, o_y=0, radius=200, step_limit = 5):
    x=np.zeros(n)
    y=np.zeros(n)
    for i in range(1,n):
        theta=2 * np.pi * random.random()
        step=round(random.uniform(0, step_limit),2)
        x[i] = x[i-1]+ step*np.cos(theta)
        y[i] = y[i-1]+ step*np.sin(theta)
        distance = (x[i] - o_x)**2 + (y[i] - o_y)**2
        if distance > radius ** 2:
            return x[0:i], y[0:i]
    return x, y

In [3]:
@njit()
def main_numba(num_packets, steps):
    packets_x = []
    packets_y = []
    for i in prange(num_packets):
        x,y = single_boundary_numba(steps)
        packets_x.append(x)
        packets_y.append(y)
    return packets_x, packets_y

In [4]:
@cuda.jit
def single_boundary_cuda(n, num_packets, x, y, o_x, o_y, radius, step_limit, packets_x, packets_y):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    print(idx)
    if idx >= num_packets:
        return

    for i in range(1,n):
        theta=2 * np.pi * random.random()
        step=round(random.uniform(0, step_limit),2)
        x[i] = x[i-1]+ step * math.cos(theta)
        y[i] = y[i-1]+ step * math.sin(theta)
        distance = (x[i] - o_x)**2 + (y[i] - o_y)**2
        
        if distance > radius ** 2:
            packets_x[idx] = x  #[0:i]
            packets_y[idx] = y   #[0:i]
            return
            
    packets_x[idx] = x
    packets_y[idx] = y

In [7]:
def main_cuda(num_packets, steps):
    packets_x = []
    packets_y = []

    # packets_x = np.zeros(num_packets)
    # packets_y = np.zeros(num_packets)
    x = np.zeros(steps)
    y = np.zeros(steps)
    single_boundary_cuda[2,4](steps, num_packets, x, y, 0, 0, 200, 5, packets_x, packets_y)

    return packets_x, packets_y



def plot_packets(packets_x, packets_y):
    plt.figure(figsize=(20,10))
    for i in range(len(packets_x)):
        plt.plot(packets_x[i],packets_y[i])
        # plt.plot(packets_x[i][0],packets_y[i][0],color='green',marker='o')
        # plt.plot(packets_x[i][-1],packets_y[i][-1],color='red',marker='o')
    plt.savefig('result.pdf')
    return

In [8]:
@cuda.jit
def gpu_print(N):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x 
    # print(cuda.blockDim, cuda.gridDim)
    if(idx < N):
        print(idx)



def main():
    print("gpu print:")
    gpu_print[2, 4](8)
    cuda.synchronize()



if __name__ == '__main__':
    x,y = main_cuda(8, 10000)
    plot_packets(x,y)
    # start = time.time()
    # x, y = main_cuda[20,25](500, 10000)
    # end = time.time()
    # print(f'Time:{end-start}')

/home/vicenteamado/.conda/envs/deepthought/lib/python3.8/site-packages/numba/cuda/compiler.py:865: NumbaPerformanceWarning: Grid size (2) < 2 * SM count (136) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


ValueError: cannot compute fingerprint of empty list